In [1]:
import os, sys, timeit

import numpy as np
import pandas as pd

from metrics import process

from tqdm import tqdm
from os.path import join
from kmeans_clustering import KMeansClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.datasets import load_boston

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
pwd = os.getcwd()
data_dir = join(pwd, 'data/')
need_normalization = False
need_standardization = False
need_pca = True
filename = 'airlines.csv'
datapath = join(data_dir, filename)
num_clusters = 10
num_iterations = 300
num_iter_exp = 20
verbose = False
data_boston = load_boston()
do_boston = False

In [3]:
# f = open('data/data.csv')
# lines = f.read()
# print(type(lines))
# new_lines = ''
# for line in lines:
#     new_lines += str(line.encode('utf-8').strip())
# print(len(new_lines))

In [4]:
df = pd.read_csv(
    datapath, 
    low_memory=False,
    encoding='utf-8',
#     sep=',',
#     header=None,
)
if do_boston is True:
    df = pd.DataFrame(
        data_boston.data, 
        columns=data_boston.feature_names,
    )

In [5]:
print(len(df))

3999


In [6]:
print(df.head())

   Balance  QualMiles  BonusMiles  BonusTrans  FlightMiles  FlightTrans  \
0    28143          0         174           1            0            0   
1    19244          0         215           2            0            0   
2    41354          0        4123           4            0            0   
3    14776          0         500           1            0            0   
4    97752          0       43300          26         2077            4   

   DaysSinceEnroll  
0             7000  
1             6968  
2             7034  
3             6952  
4             6935  


In [7]:
print(df.dtypes)

Balance            int64
QualMiles          int64
BonusMiles         int64
BonusTrans         int64
FlightMiles        int64
FlightTrans        int64
DaysSinceEnroll    int64
dtype: object


In [8]:
columns = df.columns.tolist()
cols = columns
# cols = columns[1:]
# cols = ['Survived', 'Pclass', 'Age', 'SibSp', 'Fare']
# cols = columns[:-1]
# cols = ['Passenger Count', 'Adjusted Passenger Count']
# cols = columns[1:]
# cols = columns[2:]
df = df[cols]
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.fillna(0)
df = df.replace([-np.inf, np.inf], np.nan)
df = df.dropna()
df = df.loc[:, (df != 0).any(axis=0)]
print(len(df))
print(df.dtypes)

3999
Balance            int64
QualMiles          int64
BonusMiles         int64
BonusTrans         int64
FlightMiles        int64
FlightTrans        int64
DaysSinceEnroll    int64
dtype: object


In [9]:
df = df.sample(
    frac=1.0,
    random_state=1,
)
df = df.reset_index(drop=True)

In [10]:
if need_normalization is True:
    normalizer = StandardScaler()
    tmp = normalizer.fit_transform(df)
    df = pd.DataFrame(tmp, columns=df.columns)
elif need_standardization is True:
    scaler = MinMaxScaler()
    tmp = scaler.fit_transform(df)
    df = pd.DataFrame(tmp, columns=df.columns)

In [11]:
X = df
if need_pca is True:
    pca = PCA(
        n_components=len(cols),
        svd_solver='auto',
    )
    X = pca.fit_transform(X)
else:
    X = X.to_numpy()

In [12]:
for x in X:
    for a in x:
        if np.isinf(a):
            print(x)

In [13]:
data = X
print(data[:5])
print(data.shape)

[[-3.10439687e+04  1.30402402e+04 -3.41477393e+03 -3.02483688e+02
  -6.31369423e+01 -8.61241884e-01 -3.32688308e-01]
 [-7.17712462e+04 -3.63106557e+03 -1.89696202e+03 -1.39749847e+02
  -6.28847629e+01 -1.63876152e+00 -2.54138330e-01]
 [-2.69853177e+04  9.05488883e+03 -2.41197296e+03 -3.35454954e+02
  -7.21047916e+01  2.27246369e+00 -1.19010945e-01]
 [ 6.70269297e+04 -2.40696237e+04  2.38589637e+03 -6.36093596e+02
  -1.72433953e+02  7.16784705e+00  5.05825500e-02]
 [-5.99630457e+04  1.82930922e+04 -1.64480501e+03 -3.26156743e+02
  -4.49200263e+01 -1.74754147e+00 -2.84759594e-01]]
(3999, 7)


In [14]:
models = ['coc', 'kmeans++', 'ostrovsky', 'kmeans']
# models = ['kmeans++', 'kmeans++_improved']

In [15]:
def plot_scatter(X, labels):
    fig, ax = plt.subplots()
    scatter = plt.scatter(
        X[:, 0],
        X[:, 1],
        c=labels,
    )
    handles, labels = scatter.legend_elements()
    legend = ax.legend(
        handles,
        labels,
        loc='upper right',
        title='Label',
    )
    ax.add_artist(legend)
    plt.title(model+' '+str(num_clusters)+ ' clusters')
    plt.xlabel('Component A')
    plt.ylabel('Component B')
    plt.show()

In [16]:
algorithms = {}
iterations = {}
inertias = {}
times = {}
inertias_min = {}
iters_min = {}
for model in models:
    iterations[model] = 0.0
    inertias[model] = 0.0
    times[model] = 0.0
    iters_min[model] = 0
    inertias_min[model] = 1e20
for i in tqdm(range(num_iter_exp)):
    for model in (models):
        algorithms[model] = KMeansClustering(
            n_clusters=num_clusters,
            max_iter=num_iterations,
            init=model,
            verbose=verbose,
        )
        start = timeit.default_timer()
        algorithm = algorithms[model].fit(data)
        centers = algorithm.cluster_centers_
        labels = algorithms[model].predict(data)
#         plot_scatter(X, labels)
        inertias[model] += algorithm.sse_
        inertias_min[model] = min(inertias_min[model], algorithm.sse_)
        iters_min[model] += algorithm.iter_convergence_
        end = timeit.default_timer()
        times[model] += (end-start)

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [09:47<00:00, 29.39s/it]


In [17]:
for model in models:
    print(model+', '+str(num_clusters))
    print('Average Inertia -', round(inertias[model]/num_iter_exp, 2))
    print('Minimum Inertia - ', round(inertias_min[model], 2))
    print('Time -', round(times[model]/num_iter_exp, 2))
    print('Average number of iterations', round(iters_min[model]/num_iter_exp,2))
    print('')

coc, 10
Average Inertia - 2667560530212.89
Minimum Inertia -  2627599615029.58
Time - 9.13
Average number of iterations 72.05

kmeans++, 10
Average Inertia - 2681719187656.43
Minimum Inertia -  2626435706106.56
Time - 5.41
Average number of iterations 38.45

ostrovsky, 10
Average Inertia - 2684480210031.3
Minimum Inertia -  2622281801735.52
Time - 5.52
Average number of iterations 39.6

kmeans, 10
Average Inertia - 2672710216978.34
Minimum Inertia -  2633315933118.1
Time - 9.33
Average number of iterations 74.3

